In [13]:
import requests
import numpy as np

In [14]:
url = "https://gleam-seir-api-883627921778.us-west1.run.app/prediction"

In [15]:
token = "e54d4431-5dab-474e-b71a-0db1fcb9e659"

In [16]:
headers = {"X-API-Key": token}

In [17]:
data = np.load("test.npz")
test_x = data["x"]
test_y = data["y"]

In [18]:
batch_size = 128
all_preds = []
for i in range(18):
    begin = batch_size * i
    if i == 17:
        end = None
    else:
        end = batch_size * (i + 1)

    if end is not None:
        x_batch = test_x[begin: end].tolist()
        y_batch = np.log(test_y[begin: end, 1:] + 1.)
        x0_batch = np.log(test_y[begin: end, 0] + 1.).tolist()
    else:
        x_batch = test_x[begin:].tolist()
        y_batch = np.log(test_y[begin:, 1:] + 1.)
        x0_batch = np.log(test_y[begin:, 0] + 1.).tolist()

    payload = {"x0": x0_batch, "x": x_batch}
    response = requests.post(url, headers=headers, json=payload)
    pred = np.array(response.json())
    all_preds.append(pred)

In [19]:
y_preds = np.concatenate(all_preds, axis=1)

In [21]:
y_preds.shape

(28, 2214, 24)

In [20]:
scaled_preds = np.exp(y_preds) - 1.

In [22]:
scaled_preds

array([[[1.51431709e+03, 7.09093934e+02, 4.92836134e+02, ...,
         5.77233173e+01, 3.15518544e+01, 8.45875805e+00],
        [9.10264037e+02, 4.42449527e+02, 3.10415331e+02, ...,
         4.11912667e+01, 2.28182449e+01, 6.63331850e+00],
        [5.07268071e+02, 2.53162877e+02, 1.79912467e+02, ...,
         2.72814064e+01, 1.54483319e+01, 4.96783577e+00],
        ...,
        [1.50141303e+04, 6.53711519e+03, 4.77807830e+03, ...,
         3.26921171e+02, 6.41299988e+02, 1.88487537e+02],
        [1.48709489e+04, 6.47304616e+03, 4.73303863e+03, ...,
         3.24615892e+02, 6.37631559e+02, 1.88050905e+02],
        [1.48279799e+04, 6.45176195e+03, 4.71829396e+03, ...,
         3.23220412e+02, 6.35875660e+02, 1.87925643e+02]],

       [[3.08406325e+02, 1.58643349e+02, 1.06822329e+02, ...,
         1.34736763e+01, 6.55482385e+00, 2.13362940e+00],
        [2.31585710e+02, 1.17178174e+02, 7.98405314e+01, ...,
         1.12866078e+01, 5.45218289e+00, 1.93062217e+00],
        [1.68497189e+02, 

In [23]:
scaled_preds = scaled_preds.reshape(((2214, 28, 24)))
print(scaled_preds.shape)
scaled_preds

(2214, 28, 24)


array([[[1.51431709e+03, 7.09093934e+02, 4.92836134e+02, ...,
         5.77233173e+01, 3.15518544e+01, 8.45875805e+00],
        [9.10264037e+02, 4.42449527e+02, 3.10415331e+02, ...,
         4.11912667e+01, 2.28182449e+01, 6.63331850e+00],
        [5.07268071e+02, 2.53162877e+02, 1.79912467e+02, ...,
         2.72814064e+01, 1.54483319e+01, 4.96783577e+00],
        ...,
        [1.13498627e+06, 3.96200044e+05, 2.64366889e+05, ...,
         4.83375779e+03, 2.24665941e+03, 2.14317505e+02],
        [8.14923161e+05, 2.81563996e+05, 1.86204001e+05, ...,
         4.01581842e+03, 1.84135474e+03, 1.87619776e+02],
        [8.53749950e+04, 3.14588149e+04, 2.04896288e+04, ...,
         9.00140647e+02, 4.29582035e+02, 6.16092967e+01]],

       [[6.38978785e+05, 2.22820653e+05, 1.46690872e+05, ...,
         3.53394856e+03, 1.64172992e+03, 1.77276767e+02],
        [4.71289763e+05, 1.63688220e+05, 1.07078157e+05, ...,
         2.90101956e+03, 1.33477199e+03, 1.47985720e+02],
        [1.53878126e+06, 

In [24]:
np.savez('test_pred.npz',scaled_preds)